In [ ]:
import csv
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipywidgets.widgets.trait_types import datetime_from_json
import pandas as pd
import plotly.graph_objects as go
#grafica para los nodos encontrados
class Datos:
    def __init__(self, title, department, city, property_type, latitude, longitude,
                 surface_total, surface_covered, bedrooms, bathrooms,
                 operation_type, price):
        self.title = title
        self.department = department
        self.city = city
        self.property_type = property_type
        self.latitude = latitude
        self.longitude = longitude
        self.surface_total = surface_total
        self.surface_covered = surface_covered
        self.bedrooms = bedrooms
        self.bathrooms = bathrooms
        self.operation_type = operation_type
        self.price = price

class AVLNode:
    def __init__(self, data):
        self.data = round(data,3)
        self.left = None
        self.right = None
        self.height = 1

class AVLTree:
    def __init__(self):
        self.root = None
        file_path = 'co_properties_final.csv'
        self.df = pd.read_csv(file_path)
    def _height(self, node):
        if not node:
            return 0
        return node.height

    def _get_balance(self, node):
        if not node:
            return 0
        return self._height(node.left) - self._height(node.right)

    def _rotate_left(self, z):
        y = z.right
        T2 = y.left

        y.left = z
        z.right = T2

        z.height = 1 + max(self._height(z.left), self._height(z.right))
        y.height = 1 + max(self._height(y.left), self._height(y.right))

        return y

    def _rotate_right(self, y):
        x = y.left
        T2 = x.right

        x.right = y
        y.left = T2

        y.height = 1 + max(self._height(y.left), self._height(y.right))
        x.height = 1 + max(self._height(x.left), self._height(x.right))

        return x

    def insert(self, root, data):
        if not root:
            return AVLNode(data)

        if data < root.data:
            root.left = self.insert(root.left, data)
        elif data > root.data:
            root.right = self.insert(root.right, data)
        else:
            return root  # No permitir duplicados

        root.height = 1 + max(self._height(root.left), self._height(root.right))

        balance = self._get_balance(root)

        if balance > 1:
            if data < root.left.data:
                return self._rotate_right(root)
            else:
                root.left = self._rotate_left(root.left)
                return self._rotate_right(root)
        if balance < -1:
            if data > root.right.data:
                return self._rotate_left(root)
            else:
                root.right = self._rotate_right(root.right)
                return self._rotate_left(root)

        return root
    def build_tree(self):
       datos_list=[]
       for i in range(len(self.df)):
          datos_list.append(Datos(self.df['title'][i],self.df['department'][i],self.df['city'][i],self.df['property_type'][i],self.df['latitude'][i],self.df['longitude'][i],self.df['surface_total'][i],self.df['surface_covered'][i],self.df['bedrooms'][i],self.df['bathrooms'][i],self.df['operation_type'][i],self.df['price'][i]))
       for datos in datos_list:
          metric=datos.price/datos.surface_total
          node=AVLNode(metric)
          node.data=datos

          if not self.root:
              self.root=node
          else:
              self.root=self.insert(self.root,node)

    def visualize_treeFounded(self, nodeToFound):
        fig = go.Figure()

        if self.root:
            fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', text=[str(self.root.data)],
                                     textposition="bottom center"))

            self.plot_treeFounded(fig, self.root, x=[0], y=[0], depth=1, nodeToFound=nodeToFound)
        if self.root.data in nodeToFound:
            print('Error')
            fig.add_trace(go.Scatter(x=[0],
                        y=[0],
                        mode='markers',
                        name='bla',
                        marker=dict(symbol='circle-dot',
                                        size=20,
                                        color='rgb(0,0,210)',
                                        line=dict(color='rgb(0,0,255)', width=1)
                                        ),
                        text=[str(self.root.data)],
                        hoverinfo='text',
                        opacity=0.8
                        ))
        fig.update_layout(showlegend=False)
        fig.show()
    def insertNodeUser(self):
      dictoptions = dict(self.df)
      listoptions = dictoptions.keys().tolist()
      listToInsert = [input(f" Digite el {listoptions[i]}") for i in range(len(listoptions))]
      self.insert(listToInsert[0])

    def plot_treeFounded(self, fig, node, x, y, depth, nodeToFound):

        if node.left:
            x_left = x[0] - 2 ** (6 - depth)
            y_left = y[0] - 0.1
            fig.add_trace(go.Scatter(x=[x_left], y=[y_left], mode='markers', text=[str(node.left.data)],
                                     textposition="bottom center"))
            fig.add_trace(go.Scatter(x=[x[0], x_left], y=[y[0], y_left], mode='lines'))

            if node.left.data in nodeToFound:
              fig.add_trace(go.Scatter(x=[x_left],
                        y=[y_left],
                        mode='markers',
                        name='bla',
                        marker=dict(symbol='circle-dot',
                                        size=20,
                                        color='rgb(0,0,210)',
                                        line=dict(color='rgb(0,0,255)', width=1)
                                        ),
                        text=[str(node.left.data)],
                        hoverinfo='text',
                        opacity=0.8
                        ))
            self.plot_treeFounded(fig, node.left, x=[x_left], y=[y_left], depth=depth + 1, nodeToFound=nodeToFound)

        if node.right:
            x_right = x[0] + 2 ** (6 - depth)
            y_right = y[0] - 0.1
            fig.add_trace(go.Scatter(x=[x_right], y=[y_right], mode='markers', text=[str(node.right.data)],
                                     textposition="bottom center"))
            fig.add_trace(go.Scatter(x=[x[0], x_right], y=[y[0], y_right], mode='lines'))
            if node.right.data in nodeToFound:
              fig.add_trace(go.Scatter(x=[x_right],
                          y=[y_right],
                          mode='markers',
                          name='bla',
                          marker=dict(symbol='circle-dot',
                                          size=20,
                                          color='rgb(0,0,210)',
                                          line=dict(color='rgb(0,0,255)', width=1)
                                          ),
                          text=[str(node.right.data)],
                          hoverinfo='text',
                          opacity=0.8
                          ))


            self.plot_treeFounded(fig, node.right, x=[x_right], y=[y_right], depth=depth + 1, nodeToFound=nodeToFound)

# Crear un DataFrame a partir del archivo CSV
file_path = 'co_properties_final.csv'
df = pd.read_csv(file_path)
# Inicializar el árbol AVL
avl_tree = AVLTree()

# Insertar datos en el árbol AVL basado en la métrica principal (price / surface_total)
for index, row in df.iterrows():
    metric = row['price'] / row['surface_total']
    # Insertar directamente el valor de métrica en el árbol AVL
    avl_tree.root = avl_tree.insert(avl_tree.root, metric)


In [ ]:
from ipywidgets.widgets.trait_types import datetime_from_json
import pandas as pd
import plotly.graph_objects as go
class Datos:
    def __init__(self, title, department, city, property_type, latitude, longitude,
                 surface_total, surface_covered, bedrooms, bathrooms,
                 operation_type, price):
        self.title = title
        self.department = department
        self.city = city
        self.property_type = property_type
        self.latitude = latitude
        self.longitude = longitude
        self.surface_total = surface_total
        self.surface_covered = surface_covered
        self.bedrooms = bedrooms
        self.bathrooms = bathrooms
        self.operation_type = operation_type
        self.price = price

class AVLNode:
    def __init__(self, data):
        self.data = round(data,3)
        self.left = None
        self.right = None
        self.height = 1


class AVLTree:
    def __init__(self):
        self.root = None
        file_path = 'co_properties_final.csv'
        self.df = pd.read_csv(file_path)
    def _height(self, node):
        if not node:
            return 0
        return node.height

    def _get_balance(self, node):
        if not node:
            return 0
        return self._height(node.left) - self._height(node.right)

    def _rotate_left(self, z):
        y = z.right
        T2 = y.left

        y.left = z
        z.right = T2

        z.height = 1 + max(self._height(z.left), self._height(z.right))
        y.height = 1 + max(self._height(y.left), self._height(y.right))

        return y

    def _rotate_right(self, y):
        x = y.left
        T2 = x.right

        x.right = y
        y.left = T2

        y.height = 1 + max(self._height(y.left), self._height(y.right))
        x.height = 1 + max(self._height(x.left), self._height(x.right))

        return x

    def insert(self, root, data):
        if not root:
            return AVLNode(data)

        if data < root.data:
            root.left = self.insert(root.left, data)
        elif data > root.data:
            root.right = self.insert(root.right, data)
        else:
            return root  # No permitir duplicados

        root.height = 1 + max(self._height(root.left), self._height(root.right))

        balance = self._get_balance(root)

        if balance > 1:
            if data < root.left.data:
                return self._rotate_right(root)
            else:
                root.left = self._rotate_left(root.left)
                return self._rotate_right(root)
        if balance < -1:
            if data > root.right.data:
                return self._rotate_left(root)
            else:
                root.right = self._rotate_right(root.right)
                return self._rotate_left(root)

        return root
    def build_tree(self):
       datos_list=[]
       for i in range(len(self.df)):
          datos_list.append(Datos(self.df['title'][i],self.df['department'][i],self.df['city'][i],self.df['property_type'][i],self.df['latitude'][i],self.df['longitude'][i],self.df['surface_total'][i],self.df['surface_covered'][i],self.df['bedrooms'][i],self.df['bathrooms'][i],self.df['operation_type'][i],self.df['price'][i]))
       for datos in datos_list:
          metric=datos.price/datos.surface_total
          node=AVLNode(metric)
          node.data=datos

          if not self.root:
              self.root=node
          else:
              self.root=self.insert(self.root,node)

    def visualize_tree(self):
        fig = go.Figure()

        if self.root:
            fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', text=[str(self.root.data)],
                                     textposition="bottom center"))

            self.plot_tree(fig, self.root, x=[0], y=[0], depth=1)

        fig.update_layout(showlegend=False)
        fig.show()

    def plot_tree(self, fig, node, x, y, depth):
        if node.left:
            x_left = x[0] - 2 ** (6 - depth)
            y_left = y[0] - 0.1
            fig.add_trace(go.Scatter(x=[x_left], y=[y_left], mode='markers', text=[str(node.left.data)],
                                     textposition="bottom center"))
            fig.add_trace(go.Scatter(x=[x[0], x_left], y=[y[0], y_left], mode='lines'))
            self.plot_tree(fig, node.left, x=[x_left], y=[y_left], depth=depth + 1)

        if node.right:
            x_right = x[0] + 2 ** (6 - depth)
            y_right = y[0] - 0.1
            fig.add_trace(go.Scatter(x=[x_right], y=[y_right], mode='markers', text=[str(node.right.data)],
                                     textposition="bottom center"))
            fig.add_trace(go.Scatter(x=[x[0], x_right], y=[y[0], y_right], mode='lines'))


            self.plot_tree(fig, node.right, x=[x_right], y=[y_right], depth=depth + 1)

    def visualize_treeFounded(self, nodeToFound):
        fig = go.Figure()

        if self.root:
            fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', text=[str(self.root.data)],
                                     textposition="bottom center"))

            self.plot_treeFounded(fig, self.root, x=[0], y=[0], depth=1, nodeToFound=nodeToFound)
        if self.root.data in nodeToFound:
            print('Error')
            fig.add_trace(go.Scatter(x=[0],
                        y=[0],
                        mode='markers',
                        name='bla',
                        marker=dict(symbol='circle-dot',
                                        size=20,
                                        color='rgb(0,0,210)',
                                        line=dict(color='rgb(0,0,255)', width=1)
                                        ),
                        text=[str(self.root.data)],
                        hoverinfo='text',
                        opacity=0.8
                        ))
        fig.update_layout(showlegend=False)
        fig.show()
    def insertNodeUser(self):
      dictoptions = dict(self.df)
      listoptions = dictoptions.keys().tolist()
      listToInsert = [input(f" Digite el {listoptions[i]}") for i in range(len(listoptions))]
      self.insert(listToInsert[0])

    def plot_treeFounded(self, fig, node, x, y, depth, nodeToFound):

        if node.left:
            x_left = x[0] - 2 ** (6 - depth)
            y_left = y[0] - 0.1
            fig.add_trace(go.Scatter(x=[x_left], y=[y_left], mode='markers', text=[str(node.left.data)],
                                     textposition="bottom center"))
            fig.add_trace(go.Scatter(x=[x[0], x_left], y=[y[0], y_left], mode='lines'))

            if node.left.data in nodeToFound:
              fig.add_trace(go.Scatter(x=[x_left],
                        y=[y_left],
                        mode='markers',
                        name='bla',
                        marker=dict(symbol='circle-dot',
                                        size=20,
                                        color='rgb(0,0,210)',
                                        line=dict(color='rgb(0,0,255)', width=1)
                                        ),
                        text=[str(node.left.data)],
                        hoverinfo='text',
                        opacity=0.8
                        ))
            self.plot_treeFounded(fig, node.left, x=[x_left], y=[y_left], depth=depth + 1, nodeToFound=nodeToFound)

        if node.right:
            x_right = x[0] + 2 ** (6 - depth)
            y_right = y[0] - 0.1
            fig.add_trace(go.Scatter(x=[x_right], y=[y_right], mode='markers', text=[str(node.right.data)],
                                     textposition="bottom center"))
            fig.add_trace(go.Scatter(x=[x[0], x_right], y=[y[0], y_right], mode='lines'))
            if node.right.data in nodeToFound:
              fig.add_trace(go.Scatter(x=[x_right],
                          y=[y_right],
                          mode='markers',
                          name='bla',
                          marker=dict(symbol='circle-dot',
                                          size=20,
                                          color='rgb(0,0,210)',
                                          line=dict(color='rgb(0,0,255)', width=1)
                                          ),
                          text=[str(node.right.data)],
                          hoverinfo='text',
                          opacity=0.8
                          ))


            self.plot_treeFounded(fig, node.right, x=[x_right], y=[y_right], depth=depth + 1, nodeToFound=nodeToFound)

# Crear un DataFrame a partir del archivo CSV
file_path = 'co_properties_final.csv'
df = pd.read_csv(file_path)

# Inicializar el árbol AVL
avl_tree = AVLTree()

# Insertar datos en el árbol AVL basado en la métrica principal (price / surface_total)
for index, row in df.iterrows():
    metric = row['price'] / row['surface_total']
    # Insertar directamente el valor de métrica en el árbol AVL
    avl_tree.root = avl_tree.insert(avl_tree.root, metric)

# Visualizar el árbol AVL con Plotly
avl_tree.visualize_tree()

In [ ]:
from ipywidgets.widgets.widget_selection import Dropdown
import ipywidgets as widgets
from IPython.display import display    #Importando varias clases y funciones de los módulos para crear widgets

class NodeSearcher:
    def __init__(self):  # inicializa los atributos de la clase.
        self.listaFiltered = []  #lista vacia
        self.max_selected_options = 3
        self.options = ['title', 'department', 'city', 'property_type', 'latitude', 'longitude', 'surface_total', 'surface_covered', 'bedrooms', 'bathrooms', 'operation_type', 'price']
        self.checkboxes = [widgets.Checkbox(value=False, description=opt) for opt in self.options]  #Se crean checkboxes usando las opciones disponibles.
        self.dropdowns_general = {}  #n diccionario para almacenar dropdowns.
        self.avl_tree = AVLTree()
        self.df = self.avl_tree.df

        # Crear y mostrar los dropdowns para las opciones seleccionadas

    def on_buscar_button_click(self, b):  #metodo que se activa cuando se hace clic a un boton
        for index, row in df.iterrows():
            metric = round(row['price'] / row['surface_total'], 2)
          # Insertar directamente el valor de métrica en el árbol AVL
            self.avl_tree.root = self.avl_tree.insert(self.avl_tree.root, metric)

        llaves = self.df.keys() #Guarda las llaves del diccionario

        dropdowns =self.dropdowns_general.copy()


        # Obtener las opciones seleccionadas por el usuario

        temp = {   category: [] for category in df.keys()  }  #Se crea un diccionario temporal con llaves basadas en las categorías del DataFrame
        temp['metric'] = []
        filtered_df  = pd.DataFrame(temp)
        llavesDrop = dropdowns.keys()


        for i in range(len(df.index)):
          booleanList = []              #para cada fila, se crea una lista de booleanos

          for category in llavesDrop: #bucle que itera sobre las categorías (claves), seleccionadas por el usuario
              if isinstance(dropdowns[category], list):
                  booleanList.append(df[category][i] >= dropdowns[category][0].value and df[category][i] <= dropdowns[category][1].value)
              #Si la opción no es una lista (es un valor único), se compara el valor en la fila actual de la categoría con el valor de la opción
              else:
                  booleanList.append(df[category][i] == dropdowns[category].value)

          if all(booleanList): #se verifica si todos los valores en booleanList son True
            new_row = [df[category][i] for category in df.keys() ]
            new_row.append( df['price'][i] / df['surface_total'][i] )   #se calcula y agrega la métrica
            filtered_df.loc[len(filtered_df.index)] = new_row
            filtered_df['metric'] = filtered_df['price'] / filtered_df['surface_total']



        listaFiltered = []
        # Mostrar los valores de la métrica para los nodos que cumplen con los criterios de búsqueda
        if not filtered_df.empty:  #Se verifica si el DataFrame filtrado no está vacío
            print(filtered_df['metric'])
            listaFiltered = filtered_df['metric'].tolist()
            listaFiltered = [round(i,2) for i in listaFiltered]
            print(listaFiltered)
            self.listaFiltered = listaFiltered
            print(f'Lista nueva : {self.listaFiltered}')
        else:                             #se imprime un mensaje indicando que no se encontraron nodos que cumplan con los criterios de búsqueda.
            print('No se encontraron nodos que cumplan con los criterios de búsqueda.')
    def filtrar_button(self,b):
        #contiene las descripciones de las casillas de verificación
        selected_options = [checkbox.description for checkbox in self.checkboxes if checkbox.value]
        #verifica si la cantidad de opciones seleccionadas excede el límite máximo permitido
        if len(selected_options) > self.max_selected_options:
            print('Error: Solo se pueden elegir como máximo 3 opciones.')
            return
        print(selected_options)
        #Se obtienen las claves y se almacenan en llaves.
        llaves = self.df.keys()

        for option in selected_options:
            if option not in ['latitude', 'longitude', 'surface_total', 'surface_covered', 'bedrooms', 'bathrooms', 'price']:
              dropdown_options = set(self.df[option])
               # Crear el dropdown y mostrarlo
              dropdown = widgets.Dropdown(options=dropdown_options, description=f'{option}:')
              self.dropdowns_general[option] = dropdown
              display(dropdown)

        for option in selected_options:
            if option in ['latitude', 'longitude', 'surface_total', 'surface_covered', 'bedrooms', 'bathrooms', 'price']:
                # Crear la barra deslizante para valores mínimos
                min_value = min([i for i in self.df[option]]) #list Comprenhension
                max_value = max([i for i in self.df[option]])
                min_slider = widgets.FloatSlider(value=0, min=min_value, max=max_value, description=f'{option} (min):')
                max_slider = widgets.FloatSlider(value=100, min=min_value, max=max_value, description=f'{option} (max):')
                print(option)
                #Se almacenan los sliders en el diccionario dropdowns_general bajo la clave correspondiente a la opción y luego se muestran en la interfaz.
                self.dropdowns_general[str(option)]= [min_slider, max_slider]
                # Mostrar las barras deslizantes
                display(min_slider, max_slider)
        pd.set_option('display.float_format', '{:.2f}'.format)  #que se muestren con dos decimales
        self.buscar_button = widgets.Button(description='Buscar')
        self.ejecutarInterfaz()
    def iniciarInterfaz(self): #funcion que se encarga de iniciar la interfaz gráfica mostrando botones y checkboxes.
        mostrar_button = widgets.Button(description='Mostrar Opciones')
        mostrar_button.on_click(self.filtrar_button)
        for i in self.checkboxes:  #Se itera sobre los checkboxes y se muestra cada uno de ellos
          display(i)
        display(mostrar_button)

    def ejecutarInterfaz(self):
        lista_filtrada = self.buscar_button.on_click(self.on_buscar_button_click)
        display(self.buscar_button)
    def visualize_treeFounded (self):
        print(self.listaFiltered)
        self.avl_tree.visualize_treeFounded(self.listaFiltered)
        #Llama a self.avl_tree.visualize_treeFounded(self.listaFiltered) para visualizar el árbol AVL con los datos de la lista filtrada

In [ ]:
buscador = NodeSearcher() # Esto permite utilizar los métodos y atributos definidos en la clase NodeSearcher
buscador.iniciarInterfaz()  #nicializa y muestra la interfaz gráfica (heckboxes, botón para mostrar opciones, etc)

Checkbox(value=False, description='title')

Checkbox(value=False, description='department')

Checkbox(value=False, description='city')

Checkbox(value=False, description='property_type')

Checkbox(value=False, description='latitude')

...

Dropdown(description='city:', options=('Tabio', 'Sabaneta', 'Bogotá D.C', 'Cajicá', 'Medellín', 'Floridablanca…

Dropdown(description='property_type:', options=('Otro', 'Apartamento', 'Local comercial', 'Oficina', 'Casa', '…

Dropdown(description='operation_type:', options=('Arriendo', 'Venta'), value='Arriendo')

Button(description='Buscar', style=ButtonStyle())

0   3855421.69
1   2000000.00
2   1500000.00
3   2000000.00
Name: metric, dtype: float64
[3855421.69, 2000000.0, 1500000.0, 2000000.0]
Lista nueva : [3855421.69, 2000000.0, 1500000.0, 2000000.0]


In [ ]:
buscador.visualize_treeFounded()#imprime el arbol

[3855421.69, 2000000.0, 1500000.0, 2000000.0]
